In [17]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "pierre!!"

In [18]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

2.5.5


In [61]:
input_file = "input.txt"
file = open(input_file, 'r')
values = []
for row, line in enumerate(file):
    query ="""
    WITH $line AS line, $row AS row
    WITH split(line, '')[0..-1] AS line, row
    WITH [ix IN range(0, size(line)-1) | [ix, line[ix]]] AS line, row
    UNWIND line AS col_sym
    CREATE (:Point {row:row, col:col_sym[0], sym:col_sym[1]})
    """
    gds.run_cypher(query, {"row":row, "line":line})

In [62]:
gds.run_cypher("""CREATE INDEX point_row_col IF NOT EXISTS
FOR (p:Point) ON (p.row, p.col)""")

""


In [63]:
gds.run_cypher("""MATCH (ref:Point)
CALL{ WITH ref
MATCH (east:Point WHERE east.row = ref.row and east.col = ref.col + 1)
MERGE (ref)-[:HAS_NEIGHBOR {dir:'E'}]->(east)
MERGE (ref)<-[:HAS_NEIGHBOR {dir:'W'}]-(east)
} IN TRANSACTIONS OF 1000 ROWS""")

""


In [64]:
gds.run_cypher("""MATCH (ref:Point)
CALL{ WITH ref
MATCH (south:Point WHERE south.col = ref.col and south.row = ref.row + 1)
MERGE (ref)-[:HAS_NEIGHBOR {dir:'S'}]->(south)
MERGE (ref)<-[:HAS_NEIGHBOR {dir:'N'}]-(south)
} IN TRANSACTIONS OF 1000 ROWS""")

""


In [65]:
connectable = {'|': ['N', 'S'],
'-': ['E', 'W'],
'L': ['N', 'E'],
'J': ['N', 'W'],
'7': ['S', 'W'],
'F': ['S', 'E'],
'.': [],
'S': ['N', 'S', 'E', 'W']}

In [66]:
gds.run_cypher("""
WITH $connectable AS connectable
MATCH (p:Point)-[r:HAS_NEIGHBOR WHERE NOT r.dir IN connectable[p.sym]]->()
DELETE r
""", {'connectable':connectable})

""


In [67]:
gds.run_cypher("""MATCH (a:Point)-[r:HAS_NEIGHBOR]->(b:Point)
WHERE NOT EXISTS {(b)-[:HAS_NEIGHBOR]->(a)}
SET r.toDelete = TRUE""");
gds.run_cypher("MATCH ()-[r {toDelete:TRUE}]->() DELETE r");

In [68]:
gds.run_cypher("""
MATCH (source:Point)-[r:HAS_NEIGHBOR]->(target:Point)
WITH gds.graph.project('giant_cycle', source, target) AS g
RETURN
  g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels
""")

,graph,nodes,rels
0,giant_cycle,17103,32142


In [69]:
gds.run_cypher("""
MATCH (source:Point {sym: 'S'})
CALL gds.allShortestPaths.dijkstra.stream('giant_cycle', {
    sourceNode: source
})
YIELD totalCost
RETURN max(totalCost) AS part1
""")

,part1
0,6812.0
